In [1]:
import pandas as pd
import numpy as np
import pymc as pm
import matplotlib.pyplot as plt
import arviz as az

# Load transformed data
df = pd.read_csv("../data/BrentOilPrices_transformed.csv")
df['date'] = pd.to_datetime(df['date'])
df = df.dropna(subset=['log_return'])
returns = df['log_return'].values
n = len(returns)

# Model: Change Point Detection
with pm.Model() as model:
    tau = pm.DiscreteUniform('tau', lower=0, upper=n)

    mu1 = pm.Normal('mu1', mu=0, sigma=0.05)
    mu2 = pm.Normal('mu2', mu=0, sigma=0.05)
    sigma = pm.HalfNormal('sigma', sigma=0.05)

    mu = pm.math.switch(tau > np.arange(n), mu1, mu2)

    obs = pm.Normal('obs', mu=mu, sigma=sigma, observed=returns)

    trace = pm.sample(2000, tune=1000, return_inferencedata=True, target_accept=0.95, random_seed=42)

# Summary
summary = az.summary(trace, var_names=["mu1", "mu2", "tau", "sigma"])
print(summary)

# Traceplot
az.plot_trace(trace, var_names=["mu1", "mu2", "tau"])
plt.tight_layout()
plt.show()

# Posterior of change point
tau_samples = trace.posterior['tau'].values.flatten()
best_tau = int(np.median(tau_samples))
change_date = df.iloc[best_tau]['date']
print(f"Most probable change point (median τ): {best_tau} → {change_date.date()}")

# Plot log returns with change point
plt.figure(figsize=(14,6))
plt.plot(df['date'], returns, label='Log Returns')
plt.axvline(change_date, color='red', linestyle='--', label='Change Point')
plt.title("Log Returns with Estimated Change Point")
plt.xlabel("Date")
plt.ylabel("Log Return")
plt.legend()
plt.tight_layout()
plt.show()


ModuleNotFoundError: No module named 'pandas'

In [8]:
import shutil
import pytensor
shutil.rmtree(pytensor.config.base_compiledir, ignore_errors=True)
print("PyTensor cache cleared.")


PyTensor cache cleared.
